# Starting a local neuroglancer session with FAFB dataset

### This example shows how to start a local neuroglancer session and further add neurons, synapses, neuropil meshes from a public catmaid instance

### Import neccesary library modules now

In [1]:
import navis
import fafbseg
import pymaid

In [2]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [3]:
from pyroglancer.layers import create_nglayer
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.ngviewer import openviewer, closeviewer,setviewerstate
from pyroglancer.ngspaces import create_ngspace
from pyroglancer.createconfig import createconfig

### Set configurations to fetch from data from CATMAID

In [4]:
publicurl = 'https://fafb.catmaid.virtualflybrain.org/'

In [5]:
working_rm = pymaid.CatmaidInstance(publicurl, api_token=None, project_id = 1)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO - 2021-05-05 09:29:46,488 - client - Global CATMAID instance set. Caching is ON.


### Get sample skids and neuropil meshes from CATMAID

In [6]:
sample_skids = ['40637','27295','57311','2863104','57323']

In [7]:
catmiad_neuronlist=pymaid.get_neurons(sample_skids,remote_instance = working_rm)

INFO - 2021-05-05 09:29:47,475 - utils - NumExpr defaulting to 8 threads.


In [8]:
vols = pymaid.get_volume(['AL_L', 'AL_R'], color=(255, 0, 0, .2))

In [9]:
vols['AL_R'].id = 200
vols['AL_L'].id = 300
vols

{'AL_R': <navis.Volume(name=AL_R, id=200, color=(255, 0, 0, 0.2), vertices.shape=(622, 3), faces.shape=(1240, 3))>,
 'AL_L': <navis.Volume(name=AL_L, id=300, color=(255, 0, 0, 0.2), vertices.shape=(612, 3), faces.shape=(1228, 3))>}

### Start the dataserver to host precomputed data..

In [10]:
startdataserver()

Serving data from:  /var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o
Serving directory at http://127.0.0.1:8000


### Start a basic neuroglancer local session with all FAFB configurations..

In [11]:
configdata = [dict(
             ngspace='FAFB',
             dimension=dict(x=1, y=1,z=1,units='um'),
             voxelsize=dict(x=4,y=4,z=40,units='nm'),
             layers=dict(
                    fafb_v14_clahe=dict(
                    type='image',
                    source='precomputed://gs://neuroglancer-fafb-data/fafb_v14/fafb_v14_clahe'),
                    fafb_surf=dict(
                    type='surfacemesh',
                    source='vtk://https://storage.googleapis.com/neuroglancer-fafb-data/elmr-data/FAFB.surf.vtk.gz'
                    ))
                  )]

In [12]:
configfileloc = '/Users/sri/.pyroglancer/config_temp.yml'

In [13]:
createconfig(configdata, configfileloc)

setting default config file loc


In [14]:
layer_kws = {'ngspace': 'FAFB'}

In [15]:
create_ngspace(layer_kws)

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Neuroglancer viewer created at:  http://127.0.0.1:52139/v/ab940c7baf38bd0bf1b71faea7e13545a97ce882/
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  image
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Using layout : xy-3d
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  surfacemesh
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Using layout : xy-3d


### Add skids to neuroglancer layers..

In [16]:
tmpviewer = create_nglayer(layer_kws = {'type': 'skeletons',
                                        'source': catmiad_neuronlist,
                                        'color': 'green',
                                        'alpha': 0.5})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  skeletons
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/skeletons/40637
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/skeletons/27295
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/skeletons/57311
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/skeletons/2863104
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/skeletons/57323
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/skeletons/seg_props
Using layout : xy-3d


### Add synapses to neuroglancer layers..

In [17]:
tmpviewer = create_nglayer(layer_kws = {'type': 'synapses',
                                        'source': catmiad_neuronlist})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  synapses
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/presynapses
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/postsynapses
Adding neuron:  40637
making: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/presynapses/presynapses_cell/40637
making: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/postsynapses/postsynapses_cell/40637
Adding neuron:  27295
making: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/presynapses/presynapses_cell/27295
making: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/postsynapses/postsy

127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/skeletons/info HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/skeletons/seg_props/info HTTP/1.1" 200 -


### Add neuropil meshes to neuroglancer layers..

In [18]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': [vols['AL_R'],vols['AL_L']],
                                        'name': 'neuropils','color': ['magenta', 'blue'], 'alpha': 0.3})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  volumes
mesh/200
Seg id is: 200
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/neuropils/mesh/200
mesh/300
Seg id is: 300
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/neuropils/mesh/300
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/neuropils/mesh/segment_properties
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/neuropils/mesh/segment_names
Using layout : xy-3d


### Add annotations meshes to neuroglancer layers..

In [19]:
temp_pts = pd.DataFrame([[123072, 47001, 3375]],columns=['x','y','z'])
temp_pts = pd.DataFrame([[123072, 47001, 3375], [120000, 17001, 3000]], columns=['x', 'y', 'z'])
temp_pts['description'] = ['center_pt','above_pt']

In [20]:
#plot landmarks..
tmpviewer = create_nglayer(layer_kws = {'type': 'points','name': 'landmarks',
                                        "annotationstatetype": 'precomputed',
                                        'source': temp_pts,'color': 'orange'})

config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
Layer created:  points
config file loc is at: None
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
Dimensions are in : FAFB
using default location at:  /Users/sri/.pyroglancer/config_temp.yml
using  voxel space with scale:  [4, 4, 40]
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/landmarks
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/landmarks/spatial0/0_0_0
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/landmarks/by_id/0
/private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o/precomputed/landmarks/by_id/1
Using layout : xy-3d


127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/postsynapses/info HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/presynapses/info HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/neuropils/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/landmarks/info HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/neuropils/mesh/segment_properties/info HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 09:29:49] code 404, message File not found
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/presynapses/spatial0/0_0_0 HTTP/1.1" 404 -
127.0.0.1 - - [05/May/2021 09:29:49] code 404, message File not found
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/postsynapses/spatial0/0_0_0 HTTP/1.1" 404 -
127.0.0.1 - - [05/May/2021 09:29:49] "GET /precomputed/landmarks/spatial0/0_0_0 HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2021 09:30:04] "GET /precomputed/skeletons/2863104 HTTP/1.1" 200 -
127.0.0.1 - - 

### Screenshot of the neuroglancer instance

In [21]:
tmpviewer = setviewerstate(axis_lines = False, bounding_box = False)

![alt text](pics/local_neuroglancersession.png "local neuroglancer session with neurons, synapses, neuropils etc")

### Close the viewer and dataserver

In [22]:
closeviewer()

closing already existing ng viewer


In [23]:
closedataserver()

Closing server at http://127.0.0.1:8000
Cleaning directory at /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpq1ykji9o
